# On essaie d'ouvrir les CSV fournis par Eric

Consigne : 

> trouver la variable "CYCLE" et définir la donnée correspondante à la date du 13/10/2014 13:09 par exemple

# En python pur

In [165]:
import csv 
from dateutil.parser import parse
import datetime

fname = "./jour3/RechercheCSV.csv"


def remove_duplicate(input):
    return list(set(input))


def transpose(values: dict) -> list:
    res = []
    transposed = list(zip(*values.values(), strict=True))
    for item in transposed:
        res.append(dict(zip(values.keys(), item, strict=True)))
    return res

def convert_types(values):
    """
    """
    for value in values:
        value["DATE"] = parse(value["DATE"])
        value["ACQUISITION-STARTDATE"] = parse(value["ACQUISITION-STARTDATE"])
        value["ACQUISITION-ENDDATE"] = parse(value["ACQUISITION-ENDDATE"])
    return values

def ouverture_avec_csv(fname):
    res = {}
    with open(fname) as f_csv:
        csv_handler = csv.reader(f_csv, delimiter=";")
        for line in csv_handler:
            if "[ANALYSES]" in line:
                continue
            if not line:
                continue
            header, *values = line
            res[header] = values
            
    return res

def traitement(res_converted):
    res = []
    for element in res_converted:
        if datetime.datetime(2015, 5, 1) < element["DATE"] < datetime.datetime(2017, 5, 10):
            res.append(element["CYCLE"])
    
    return remove_duplicate(res)

class CSV:
    def __init__(self, fname):
        self.fname = fname
        self.data = convert_types(transpose(ouverture_avec_csv(fname)))

    def get_values_between_dates(self, start, end):
        res = []
        for element in self.data:
            if start < element["DATE"] < end:
                res.append(element["CYCLE"])
        return remove_duplicate(res)

In [166]:
# sans OOP
res = ouverture_avec_csv(fname)
res_transpose = transpose(res)
res_converted = convert_types(res_transpose)
print(traitement(res_converted))

['TRI134', 'TRI133', 'TRI135']


In [167]:
# avec OOP
c = CSV(fname)
c.get_values_between_dates(datetime.datetime(2015, 5, 1), datetime.datetime(2017, 5, 10))

['TRI134', 'TRI133', 'TRI135']

## Annexe : transposées de matrice 

In [151]:
a = [
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4],
]

print(a[3][0], a[0][3])

1 4


In [152]:
b = list(zip(*a, strict=True))
print(b)
print(b[3][0], b[0][3])

[(1, 1, 1, 1), (2, 2, 2, 2), (3, 3, 3, 3), (4, 4, 4, 4)]
4 1


# Avec Pandas

In [229]:
import pandas as pd 

df = pd.read_csv(fname, skiprows=1, sep=";")
df = df.T
df.reset_index(inplace=True)
df.rename(columns=df.iloc[0], inplace=True)
df.drop(0, inplace=True)

pd.to_datetime(df.DATE, format="%Y/%m/%d %H:%M:%S")

ValueError: time data "15/09/14 11:49:41" doesn't match format "%Y/%m/%d %H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [221]:
df

,DATE,ACQUISITION-STARTDATE,ACQUISITION-ENDDATE,STATUS,SETUP_NAME,CYCLE,TEST-LABEL
1,15/09/14 11:49:41,15/09/14 11:49:41,15/09/14 12:09:51,DIAGNOSIS,DEFAULT,TRI133,011
2,01/10/14 16:10:02,01/10/14 16:10:02,01/10/14 16:30:13,DIAGNOSIS,DEFAULT,TRI133,012
3,06/10/14 16:31:56,06/10/14 16:31:56,06/10/14 16:52:07,DIAGNOSIS,DEFAULT,TRI133,013
4,13/10/14 13:09:54,13/10/14 13:09:54,13/10/14 13:30:04,DIAGNOSIS,DEFAULT,TRI133,014
5,20/10/14 12:04:48,20/10/14 12:04:48,20/10/14 12:24:58,DIAGNOSIS,DEFAULT,TRI133,015
...,...,...,...,...,...,...,...
258,24/07/23 09:48:33,24/07/23 09:48:33,24/07/23 10:08:38,DIAGNOSIS,V2 MAJ 26-11-2019,TRI140,063
259,31/07/23 10:59:08,31/07/23 10:59:08,31/07/23 11:19:19,DIAGNOSIS,V2 MAJ 26-11-2019,TRI140,064
260,07/08/23 10:08:52,07/08/23 10:08:52,07/08/23 10:28:58,DIAGNOSIS,V2 MAJ 26-11-2019,TRI140,067
261,14/08/23 11:09:11,14/08/23 11:09:11,14/08/23 11:29:17,DIAGNOSIS,V2 MAJ 26-11-2019,TRI140,068
